In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

from astropy.io import fits
from astropy.table import Table
from specutils import Spectrum1D
from scipy.optimize import curve_fit
from astropy.stats import sigma_clip
from glob import glob

In [29]:
# Load in RU Lupi data
spectrum_files = glob(os.path.join('HST','mastDownload','HST','*','*x1dsum.fits'))
spectrum_files

['HST/mastDownload/HST/lbgj02040/lbgj02040_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02050/lbgj02050_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02060/lbgj02060_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02070/lbgj02070_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02080/lbgj02080_x1dsum.fits',
 'HST/mastDownload/HST/lbgj02090/lbgj02090_x1dsum.fits',
 'HST/mastDownload/HST/lbgj020a0/lbgj020a0_x1dsum.fits',
 'HST/mastDownload/HST/lbgj020b0/lbgj020b0_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53040/lbgj53040_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53050/lbgj53050_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53060/lbgj53060_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53070/lbgj53070_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53080/lbgj53080_x1dsum.fits',
 'HST/mastDownload/HST/lbgj53090/lbgj53090_x1dsum.fits',
 'HST/mastDownload/HST/lbgj530a0/lbgj530a0_x1dsum.fits',
 'HST/mastDownload/HST/lbgj530b0/lbgj530b0_x1dsum.fits',
 'HST/mastDownload/HST/leit1c030/leit1c030_x1dsum.fits',
 'HST/mastDownload/HST/leit1c04

In [31]:
spectrum_header = [fits.getheader(file,ext=0) for file in spectrum_files]
spectrum_x1d = []
for file in spectrum_files:
    print(file)
    spectrum_x1d.append(Table.read(file, hdu=1))

OSError: Empty or corrupt FITS file

In [30]:
spec = fits.open('HST/mastDownload/HST/leit1m040/leit1m040_x1dsum.fits')[1]
spec.data

OSError: Empty or corrupt FITS file

In [9]:
spectrum_x1d